# This program connects to Box.com, gets all of the folders and images and stores information into the database.

https://developer.box.com/en/guides/authentication/oauth2/with-sdk/

In [ ]:
!pip install boxsdk
!pip install auth
!pip install redis
!pip install mysql.connector
!pip install requests

In [1]:
from boxsdk import OAuth2

import json
#Set the file we want to use for authenticating a Box app
#The json file stores the client_id and client_secret so we don't have it in the code.
# The json file looks like this:
#{
#"client_id":"___the_codes_for_client_id___",
#"client_secret":"___the_codes_for_client_secret___"
#}

oauth_settings_file = 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\box_app_test.json'
with open(oauth_settings_file, "r") as read_file:
    oauth_data = json.load(read_file)
print(oauth_data["client_id"])
print(oauth_data["client_secret"])

oauth = OAuth2(
    client_id=oauth_data["client_id"],
    client_secret=oauth_data["client_secret"]
)

auth_url, csrf_token = oauth.get_authorization_url('https://jeffblackadar.ca')
print("Click on this:")
print(auth_url)
print(csrf_token)
print("Copy the code that follows code= in the URL.  Paste it into the oauth.authenticate('___the_code___') below.  Be quick, the code lasts only a few seconds.")


rvy5o5ic72c00ldi8vpcqmof0t7h6w85
VWJQ1LTracitLw5HfG8Q0TKplBogFEIT
Click on this:
https://account.box.com/api/oauth2/authorize?state=box_csrf_token_s71i4Aw24kTmBzKJ&response_type=code&client_id=rvy5o5ic72c00ldi8vpcqmof0t7h6w85&redirect_uri=https%3A%2F%2Fjeffblackadar.ca
box_csrf_token_s71i4Aw24kTmBzKJ
Copy the code that follows code= in the URL.  Paste it into the oauth.authenticate('___the_code___') below.  Be quick, the code lasts only a few seconds.


In [2]:
from boxsdk import Client

# Make sure that the csrf token you get from the `state` parameter
# in the final redirect URI is the same token you get from the
# get_authorization_url method to protect against CSRF vulnerabilities.
#assert 'THE_CSRF_TOKEN_YOU_GOT' == csrf_token
access_token, refresh_token = oauth.authenticate('RPLHGWYys5MXquaMgYqKqMT093RFSV8Y')
client = Client(oauth)

In [3]:
def process_subfolder(client, folder_id, folder_name,cursor_insert):
    print("this folder: "+folder_name)
    items = client.folder(folder_id=folder_id).get_items()
    for item in items:
        print('{0} {1} is named "{2}"'.format(item.type.capitalize(), item.id, item.name))
        if(item.type.capitalize()=="Folder"):
            process_subfolder(client, item.id,folder_name+"/"+item.name,cursor_insert)
        if(item.type.capitalize()=="File"):
            #print(item)
            print('{0} is named "{1}" path: {2}'.format(item.id, item.name, folder_name+"/"+item.name))
            
            insert_image = ("INSERT INTO tbl_box_images (`id_box_file`,`id_box_folder`,`folder`,`file_name`,`img_src`,`img_url`) VALUES (%s,%s,%s,%s,%s,%s);")
            
            cursor_insert.execute(insert_image, (item.id, folder_id, folder_name, item.name, folder_name+"/"+item.name, "https://pompeiiinpictures.com/pompeiiinpictures"+folder_name+"/"+item.name))
            mariadb_connection.commit()
    return




In [4]:
import requests
import shutil
from pathlib import Path
base_dir = 'C:\\a_orgs\\carleton\\hist3814\\R\\pompeii\\box'


import mysql.connector as mariadb

mariadb_connection = mariadb.connect(option_files='C:\\ProgramData\\MySQL\\MySQL Server 8.0\\webpage_images.cnf', option_groups="webpage_images")

cursor_insert = mariadb_connection.cursor(buffered=True)

process_subfolder(client, '0',"", cursor_insert)
    
mariadb_connection.close()    


this folder: 
Folder 98208868103 is named "lop"
this folder: /lop
Folder 98436941432 is named "1963"
this folder: /lop/1963
File 588118649408 is named "Elizabeth II young 2019-08-10 15_41_20.591925.jpg"
588118649408 is named "Elizabeth II young 2019-08-10 15_41_20.591925.jpg" path: /lop/1963/Elizabeth II young 2019-08-10 15_41_20.591925.jpg
File 588114839194 is named "Elizabeth II young 2019-08-10 15_41_52.188758.jpg"
588114839194 is named "Elizabeth II young 2019-08-10 15_41_52.188758.jpg" path: /lop/1963/Elizabeth II young 2019-08-10 15_41_52.188758.jpg
File 587019307270 is named "eII2900.png"
587019307270 is named "eII2900.png" path: /lop/eII2900.png
File 587019495720 is named "eII2901.png"
587019495720 is named "eII2901.png" path: /lop/eII2901.png
File 587019193229 is named "eII2903.png"
587019193229 is named "eII2903.png" path: /lop/eII2903.png
